#Final B3 - Grupo 1

Análisis de DF de clientes bancarios para detectar bajas en un periodo de dos meses aplicando modelos predictivos.

Notebook 3/3 = Evaluación

Integrantes:

Pascuccio, José (1017995)

Bugarin Carlos Maximiliano (82125)

Calda Ariel Gustavo (1158887)

Capello Juan Andrés (1158799)

Ricci Cecilia (1160479)

Siracusa Angel Jonatán (1160480)

Nievas Ivana (1157295)




##Importación de librerías

In [1]:
import pandas as pd
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/B3/Final'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Carga del DF 
Evaluaremos tomando foto mes 201404 para predecir que clientes son los candidatos para la campaña de marketing para evitar la baja de los mismos. Se filtra el DF y se aplican las mismas limpiezas y transformaciones que se aplicaron al DF antes de entrenar los modelos


In [3]:
df = pd.read_csv(path +'/MESES_hasta_201404.zip')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,47,81,83) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# Filtro el foto_mes 201404
df2=df[df.foto_mes == 201404].copy()
df2.head()

,numero_de_cliente,foto_mes,marketing_activo_ultimos90dias,cliente_vip,cliente_sucursal,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_tconsumos,Visa_tadelantosefectivo,Visa_mpagominimo
7,3333,201404,0,0,455,0,0,-1351755.68,-1.069960e+07,276455.04,...,.,.,.,.,.,.,.,S,S,.
8,3333,201404,0,0,455,0,0,2451895.62,-1.524554e+08,-569108.17,...,.,.,.,.,.,.,.,S,S,.
24,5609788,201404,1,0,224,62,239,-83.11,8.931040e+03,395.03,...,0,20140430,0,-5412.65,0,19970317,266.8,S,N,3954.6
40,5609981,201404,1,0,227,65,198,379.81,4.411550e+03,144.07,...,0,20140430,0,-8795.27,0,20010308,776.88,S,N,0
56,5610023,201404,1,0,536,41,93,491.06,6.913570e+03,843.05,...,0,20140430,0,-6901.64,0,20060802,708.2,S,N,3077.1


## Aplicamos las mismas limpiezas y transformaiones que en la preparacion del DF analizado


In [5]:
mask = ((df2['foto_mes'].astype('str')==df2['Master_fechaalta'].str[:6]) | (df2['foto_mes'].astype('str')==df2['Visa_fechaalta'].str[:6])) 
df2['alta_tarj_mes'] = np.where(mask, 1, 0)
df2 = df2.drop(['Master_fechaalta','Visa_fechaalta','Visa_Fvencimiento', 'Master_Fvencimiento'], axis='columns')

In [6]:
mask = (df2['Master_fultimo_cierre']=='.')
print(df2[mask]['Master_fultimo_cierre'].head(10))
df2.loc[df2[mask].index, ['Master_fultimo_cierre']]=df2[mask]['foto_mes']*100 + 15
print(df2[mask]['Master_fultimo_cierre'].head(10))

7      .
8      .
108    .
172    .
383    .
447    .
737    .
927    .
949    .
981    .
Name: Master_fultimo_cierre, dtype: object
7      20140415
8      20140415
108    20140415
172    20140415
383    20140415
447    20140415
737    20140415
927    20140415
949    20140415
981    20140415
Name: Master_fultimo_cierre, dtype: object


In [7]:
mask = (df2['Visa_fultimo_cierre']=='.')
print(df2[mask]['Visa_fultimo_cierre'].head(10))
df2.loc[df2[mask].index, ['Visa_fultimo_cierre']]=df2[mask]['foto_mes']*100 + 15
print(df2[mask]['Visa_fultimo_cierre'].head(10))

7       .
8       .
108     .
447     .
769     .
927     .
949     .
1029    .
1177    .
1321    .
Name: Visa_fultimo_cierre, dtype: object
7       20140415
8       20140415
108     20140415
447     20140415
769     20140415
927     20140415
949     20140415
1029    20140415
1177    20140415
1321    20140415
Name: Visa_fultimo_cierre, dtype: object


In [8]:
df2['marketing_activo_ultimos90dias'] = df2['marketing_activo_ultimos90dias'].astype(int)

In [9]:
df2 = df2.drop(['marketing_activo_ultimos90dias', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'tpaquete_premium', 'tpaquete3', 'tpaquete5', 'tpaquete8', 'mcuenta_corriente_dolares', 'mbonos_gobierno', 'mmonedas_extranjeras', 'minversiones_otras', 'tplan_sueldo', 'mplan_sueldo', 'ccuenta_descuentos', 'mcuenta_descuentos', 'thomebanking', 'chomebanking_transacciones', 'tautoservicio', 'cautoservicio_transacciones', 'tcajas_consultas'], axis='columns')

In [10]:
# Esto solo se puede hacer en las columnas binarias (de 2 valores)
for c in df2.columns:
    a = df2[c].unique().tolist()
    if (len(a)==2) & (a.count('S')==1):
        df2[c] = np.where(df2[c]=='S', 1, 0)

In [11]:
df2 = df2.drop(['Master_Finiciomora' , 'Visa_Finiciomora'] , axis='columns')

## Transformación del DF

In [12]:
print(df2.shape)        
df2= pd.get_dummies(df2, columns=["Master_marca_atraso","Master_cuenta_estado", \
                                                      "Visa_marca_atraso","Visa_cuenta_estado"], \
                              prefix=["MMA","MCE","VMA","VCE"])
print(df2.shape)

(184541, 143)
(184541, 155)


In [13]:
print(df2.shape)   
drop_cols= ['MMA_.', 'MCE_.','VMA_.','VCE_.']
df2 = df2.drop(drop_cols , axis='columns')
print(df2.shape) 

(184541, 155)
(184541, 151)


In [14]:
df2['mprestamos_hipotecarios'].replace({".": "0"}, inplace=True)
df2['mcuenta_debitos_automaticos'].replace({".": "0"}, inplace=True)
df2['mttarjeta_visa_debitos_automatic'].replace({".": "0"}, inplace=True)
df2['mttarjeta_master_debitos_automat'].replace({".": "0"}, inplace=True)
df2['mpagodeservicios'].replace({".": "0"}, inplace=True)
df2['mpagomiscuentas'].replace({".": "0"}, inplace=True)
df2['Master_mfinanciacion_limite'].replace({".": "0"}, inplace=True)
df2['Master_msaldototal'].replace({".": "0"}, inplace=True)
df2['Master_msaldopesos'].replace({".": "0"}, inplace=True)
df2['Master_msaldodolares'].replace({".": "0"}, inplace=True)
df2['Master_mconsumospesos'].replace({".": "0"}, inplace=True)
df2['Master_mconsumosdolares'].replace({".": "0"}, inplace=True)
df2['Master_mlimitecompra'].replace({".": "0"}, inplace=True)
df2['Master_madelantopesos'].replace({".": "0"}, inplace=True)
df2['Master_madelantodolares'].replace({".": "0"}, inplace=True)
df2['Master_mpagado'].replace({".": "0"}, inplace=True)
df2['Master_mpagospesos'].replace({".": "0"}, inplace=True)
df2['Master_mpagosdolares'].replace({".": "0"}, inplace=True)
df2['Master_mconsumototal'].replace({".": "0"}, inplace=True)
df2['Master_mpagominimo'].replace({".": "0"}, inplace=True)
df2['Visa_mfinanciacion_limite'].replace({".": "0"}, inplace=True)
df2['Visa_msaldototal'].replace({".": "0"}, inplace=True)
df2['Visa_msaldopesos'].replace({".": "0"}, inplace=True)
df2['Visa_msaldodolares'].replace({".": "0"}, inplace=True)
df2['Visa_mconsumospesos'].replace({".": "0"}, inplace=True)
df2['Visa_mconsumosdolares'].replace({".": "0"}, inplace=True)
df2['Visa_mlimitecompra'].replace({".": "0"}, inplace=True)
df2['Visa_madelantopesos'].replace({".": "0"}, inplace=True)
df2['Visa_madelantodolares'].replace({".": "0"}, inplace=True)
df2['Visa_mpagado'].replace({".": "0"}, inplace=True)
df2['Visa_mpagospesos'].replace({".": "0"}, inplace=True)
df2['Visa_mpagosdolares'].replace({".": "0"}, inplace=True)
df2['Visa_mconsumototal'].replace({".": "0"}, inplace=True)
df2['Visa_mpagominimo'].replace({".": "0"}, inplace=True)
df2['mtarjeta_visa_descuentos'].replace({".": "0"}, inplace=True)
df2['mtarjeta_master_descuentos'].replace({".": "0"}, inplace=True)


In [15]:
df2['Master_mfinanciacion_limite'] = df2['Master_mfinanciacion_limite'].astype(float)
df2['Master_msaldototal'] = df2['Master_msaldototal'].astype(float)
df2['Master_msaldopesos'] = df2['Master_msaldopesos'].astype(float)
df2['Master_msaldodolares'] = df2['Master_msaldodolares'].astype(float)
df2['Master_mconsumospesos'] = df2['Master_mconsumospesos'].astype(float)
df2['Master_mconsumosdolares'] = df2['Master_mconsumosdolares'].astype(float)
df2['Master_mlimitecompra'] = df2['Master_mlimitecompra'].astype(float)
df2['Master_madelantopesos'] = df2['Master_madelantopesos'].astype(float)
df2['Master_madelantodolares'] = df2['Master_madelantodolares'].astype(float)
df2['Master_mpagado'] = df2['Master_mpagado'].astype(float)
df2['Master_mpagospesos'] = df2['Master_mpagospesos'].astype(float)
df2['Master_mpagosdolares'] = df2['Master_mpagosdolares'].astype(float)
df2['Master_mconsumototal'] = df2['Master_mconsumototal'].astype(float)
df2['Master_mpagominimo'] = df2['Master_mpagominimo'].astype(float)
df2['Visa_mfinanciacion_limite'] = df2['Visa_mfinanciacion_limite'].astype(float)
df2['Visa_msaldototal'] = df2['Visa_msaldototal'].astype(float)
df2['Visa_msaldopesos'] = df2['Visa_msaldopesos'].astype(float)
df2['Visa_msaldodolares'] = df2['Visa_msaldodolares'].astype(float)
df2['Visa_mconsumospesos'] = df2['Visa_mconsumospesos'].astype(float)
df2['Visa_mconsumosdolares'] = df2['Visa_mconsumosdolares'].astype(float)
df2['Visa_mlimitecompra'] = df2['Visa_mlimitecompra'].astype(float)
df2['Visa_madelantopesos'] = df2['Visa_madelantopesos'].astype(float)
df2['Visa_madelantodolares'] = df2['Visa_madelantodolares'].astype(float)
df2['Visa_mpagado'] = df2['Visa_mpagado'].astype(float)
df2['Visa_mpagospesos'] = df2['Visa_mpagospesos'].astype(float)
df2['Visa_mpagosdolares'] = df2['Visa_mpagosdolares'].astype(float)
df2['Visa_mconsumototal'] = df2['Visa_mconsumototal'].astype(float)
df2['Visa_mpagominimo'] = df2['Visa_mpagominimo'].astype(float)
df2['mprestamos_hipotecarios'] = df2['mprestamos_hipotecarios'].astype(float) 
df2['mcuenta_debitos_automaticos'] = df2['mcuenta_debitos_automaticos'].astype(float)
df2['mttarjeta_visa_debitos_automatic'] = df2['mttarjeta_visa_debitos_automatic'].astype(float) 
df2['mttarjeta_master_debitos_automat'] = df2['mttarjeta_master_debitos_automat'].astype(float)
df2['mpagodeservicios'] = df2['mpagodeservicios'].astype(float) 
df2['mpagomiscuentas'] = df2['mpagomiscuentas'].astype(float) 
df2['mtarjeta_visa_descuentos'] = df2['mtarjeta_visa_descuentos'].astype(float)
df2['mtarjeta_master_descuentos'] = df2['mtarjeta_master_descuentos'].astype(float)


In [16]:
df2 = df2.drop(['Master_fultimo_cierre' , 'Visa_fultimo_cierre'] , axis='columns')


In [17]:
print(df2.shape)

(184541, 149)


##Aplicamos el modelo elegido
De acuerdo a lo analizado en el notebook 2, levantamos el modelo elegido que guardamos en disco y aplicamos las predicciones

In [18]:
from joblib import dump, load
ModeloGB = load(path + '/GBoost1.joblib')


In [19]:
y_pred = ModeloGB.predict(df2)

In [20]:
predicciones = ModeloGB.predict_proba(df2)[:,1]

In [23]:
predicciones_s = pd.Series(predicciones, index=df2.index.tolist())
y_pred_s = pd.Series(y_pred, index=df2.index.tolist())

In [24]:
finalDf= pd.concat([
df2['numero_de_cliente'],
pd.DataFrame(y_pred_s, columns=['Pred']),
pd.DataFrame(predicciones_s, columns=['Proba'])
], axis=1)
finalDf

,numero_de_cliente,Pred,Proba
7,3333,0,0.016756
8,3333,0,0.063827
24,5609788,0,0.000824
40,5609981,0,0.016915
56,5610023,0,0.000293
...,...,...,...
2694625,48914938,0,0.015928
2694626,48915521,0,0.022209
2694627,48922297,0,0.001497
2694628,48926901,0,0.047055


In [25]:
finalDf['Proba'].astype(float)

7          0.016756
8          0.063827
24         0.000824
40         0.016915
56         0.000293
             ...   
2694625    0.015928
2694626    0.022209
2694627    0.001497
2694628    0.047055
2694629    0.047055
Name: Proba, Length: 184541, dtype: float64

##Seleccion de los candidatos a la campaña de marketing
Se aplica al DF anterior el Threshold obtenido con el modelo seleccionado en el notebook 2 (0.021), filtrando los clientes correspondientes
 


In [26]:
df_elegidos = finalDf[finalDf['Proba'] > 0.021].copy()
df_elegidos.head()

,numero_de_cliente,Pred,Proba
8,3333,0,0.063827
108,5610328,0,0.070562
351,5613512,0,0.021997
447,5614357,0,0.021247
1321,5619428,0,0.060139


In [27]:
df_elegidos.shape

(7556, 3)

In [28]:
#carta 1591 sobre 45496
print('Porcentaje obtenido en el test del notebook2',1590/45496 * 100)
print('Porcentaje obtenido en la estimacion actual',7556/184541 * 100)


Porcentaje obtenido en el test del notebook2 3.4948127307895196
Porcentaje obtenido en la estimacion actual 4.094483068803139


La lista de los clientes para la campaña de marketing quedan en el archivo CampaniaClientes_201406.csv


In [29]:
df_elegidos['numero_de_cliente'].to_csv(path + '/CampaniaClientes_201406.csv')